In [51]:
import pandas as pd
import numpy as np
import midterm1_helper as mh
import matplotlib.pyplot as plt
import re
import cmds.portfolio_management_helper as pmh

In [60]:
df_info = pd.read_excel('data/midterm_1_data_2023.xlsx',sheet_name='info')
df_excess_returns = pd.read_excel('data/midterm_1_data_2023.xlsx',sheet_name='excess returns')
df_spy = pd.read_excel('data/midterm_1_data_2023.xlsx',sheet_name='spy')

In [61]:
df_excess_returns = df_excess_returns.rename(columns={'date':'Date'})
df_excess_returns.set_index('Date', inplace=True)
df_excess_returns.head()

,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,XOM
Date,,,,,,,
2016-01-15,-0.0061,-0.0334,-0.0686,-0.0929,-0.0358,-0.0363,0.0309
2016-01-22,0.0453,0.0266,0.0471,0.0505,0.0503,-0.0108,-0.0119
2016-01-29,-0.0517,0.0421,-0.0272,0.0180,0.0098,-0.0675,0.0052
2016-02-05,-0.0362,-0.0968,-0.1519,-0.1050,-0.0830,-0.1569,0.0213
2016-02-12,-0.0079,-0.0008,0.0023,-0.0341,-0.0031,-0.0787,0.0135


In [62]:
pmh.calc_summary_statistics(
    df_excess_returns,
    annual_factor=52,
    provided_excess_returns=True,
    keep_columns=['Annualized Vol', 'Annualized Mean', 'Annualized Sharpe']
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
AAPL,0.3194,0.2839,1.1252
MSFT,0.2881,0.2402,1.1993
AMZN,0.2395,0.3104,0.7715
NVDA,0.6507,0.4681,1.3900
GOOGL,0.1933,0.2742,0.7050
TSLA,0.5697,0.6070,0.9386
XOM,0.1242,0.3116,0.3986


In [63]:
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [64]:
performance_summary(df_excess_returns, 52)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
AAPL,0.3194,0.2839,1.1252,-0.3343,2.6722,-0.0523,-0.0856,-0.1906,0.1436,-0.3721,2018-10-05,2019-01-04,2019-11-08
MSFT,0.2881,0.2402,1.1993,-0.3592,1.7372,-0.0494,-0.0716,-0.1505,0.1042,-0.2995,2020-02-14,2020-03-20,2020-07-03
AMZN,0.2395,0.3104,0.7715,-0.2106,1.7463,-0.0619,-0.0961,-0.1519,0.1561,-0.4681,2021-07-09,2023-01-06,NaT
NVDA,0.6507,0.4681,1.3900,0.4257,2.2444,-0.0838,-0.1194,-0.2102,0.3326,-0.5923,2021-11-19,2022-10-14,2023-05-19
GOOGL,0.1933,0.2742,0.7050,0.0420,1.1436,-0.0557,-0.0784,-0.1355,0.1493,-0.3483,2022-03-25,2023-01-06,NaT
TSLA,0.5697,0.6070,0.9386,0.4415,1.5274,-0.1225,-0.1553,-0.2850,0.3349,-0.6822,2021-11-05,2023-01-06,NaT
XOM,0.1242,0.3116,0.3986,0.0979,3.1295,-0.0617,-0.0973,-0.1753,0.1842,-0.6714,2016-12-16,2020-03-20,2022-03-11


In [65]:
tangency = pmh.calc_tangency_weights(df_excess_returns)
tangency

,Tangency Weights
AAPL,0.3226
MSFT,0.7875
AMZN,-0.2286
NVDA,0.4960
GOOGL,-0.5027
TSLA,0.1060
XOM,0.0193


In [66]:
pmh.calc_summary_statistics(
    pmh.calc_tangency_weights(df_excess_returns, return_port_ret = True),
    annual_factor=52,
    provided_excess_returns=True,
    keep_columns=['Annualized Vol', 'Annualized Mean', 'Annualized Sharpe']
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
Tangency Portfolio,0.5635,0.3584,1.5724


In [71]:
best_worst_sharpe=(
    pmh.calc_summary_statistics(
        df_excess_returns,
        annual_factor=52,
        provided_excess_returns=True,
        keep_columns=['Annualized Sharpe']
    )
    .sort_values("Annualized Sharpe")
    .reset_index()
    .loc[lambda df: df.index.isin([0, len(df.index)-1])]
    .rename(columns={"index": "Asset"})
    .assign(Label=["Worst Sharpe", "Best Sharpe"])
)
best_worst_sharpe

,Asset,Annualized Sharpe,Label
0,XOM,0.3986,Worst Sharpe
6,NVDA,1.3900,Best Sharpe


In [72]:
tangency.loc[best_worst_sharpe.loc[0, 'Asset']]

Tangency Weights   0.0193
Name: XOM, dtype: float64

In [74]:
tangency.loc[best_worst_sharpe.loc[6, 'Asset']]

Tangency Weights   0.4960
Name: NVDA, dtype: float64

In [75]:
portfolio_tangency = pmh.calc_tangency_weights(df_excess_returns.loc[:'2022-12-30'])
portfolio_tangency

,Tangency Weights
AAPL,0.3106
MSFT,1.0731
AMZN,-0.2591
NVDA,0.3801
GOOGL,-0.7515
TSLA,0.1016
XOM,0.1453


In [76]:
n_assets = len(df_excess_returns.columns)
EW_portfolio = pd.DataFrame(index = df_excess_returns.columns)
EW_portfolio.loc[:, 'EW Weights'] = 1/n_assets
EW_portfolio

,EW Weights
AAPL,0.1429
MSFT,0.1429
AMZN,0.1429
NVDA,0.1429
GOOGL,0.1429
TSLA,0.1429
XOM,0.1429


In [77]:
df_variance_dict = df_excess_returns.std().map(lambda x: x ** 2).to_dict()
df_inv_variance_dict = {asset: 1 / variance for asset, variance in df_variance_dict.items()}
RP_portfolio = pd.DataFrame(index = df_excess_returns.columns)
for col in df_variance_dict:
    RP_portfolio.loc[col, 'EW Weights'] = df_variance_dict[col]
RP_portfolio

,EW Weights
AAPL,0.0015
MSFT,0.0011
AMZN,0.0019
NVDA,0.0042
GOOGL,0.0014
TSLA,0.0071
XOM,0.0019


In [78]:
Reg_Portfolio = pmh.calc_tangency_weights(df_excess_returns, return_port_ret=True, cov_mat=1/3, name="Regularized")
Reg_Portfolio


,Regularized Portfolio
date,
2016-01-15,-0.0399
2016-01-22,0.0334
2016-01-29,-0.0032
2016-02-05,-0.0880
2016-02-12,-0.0161
...,...
2023-06-16,0.0488
2023-06-23,-0.0097
2023-06-30,0.0234


In [79]:
EW_portfolio.T @ df_excess_returns.mean(),

(EW Weights   0.0066
 dtype: float64,)

In [80]:
IN_SAMPLE_END_DATE = "'2022-12-30'"
OUT_OF_SAMPLE_START_DATE = "2023-01-06"

in_sample_assets_excess_returns = df_excess_returns.loc[:IN_SAMPLE_END_DATE]
out_of_sample_assets_excess_returns = df_excess_returns.loc[OUT_OF_SAMPLE_START_DATE:]

In [81]:
n_assets = len(df_excess_returns.columns)

# Equal Weights
in_sample_weights_equal = pd.DataFrame(
    data=[[1 / n_assets] for _ in range(n_assets)],
    columns=["Equal Weights"],
    index=in_sample_assets_excess_returns.columns
)
in_sample_weights_equal

,Equal Weights
AAPL,0.1429
MSFT,0.1429
AMZN,0.1429
NVDA,0.1429
GOOGL,0.1429
TSLA,0.1429
XOM,0.1429


In [84]:
# Risk Parity
in_sample_asset_variance_dict = in_sample_assets_excess_returns.std().map(lambda x: x ** 2).to_dict()
in_sample_asset_inv_variance_dict = {asset: 1 / variance for asset, variance in in_sample_asset_variance_dict.items()}
in_sample_weights_risk_parity = pd.DataFrame(in_sample_asset_inv_variance_dict, index=["Risk Parity Weights"]).transpose()
in_sample_weights_risk_parity

,Risk Parity Weights
AAPL,617.9459
MSFT,925.5907
AMZN,537.3740
NVDA,241.2644
GOOGL,713.2440
TSLA,144.1950
XOM,532.3875


In [86]:
in_sample_weights_regularized = pmh.calc_tangency_weights(in_sample_assets_excess_returns, cov_mat=1/3, name="Regularized")
in_sample_weights_regularized


,Regularized Weights
AAPL,0.2373
MSFT,0.3308
AMZN,0.0472
NVDA,0.1968
GOOGL,0.0114
TSLA,0.0902
XOM,0.0864


In [87]:
in_sample_weights_tangency = pmh.calc_tangency_weights(in_sample_assets_excess_returns)
in_sample_weights_tangency

,Tangency Weights
AAPL,0.3106
MSFT,1.0731
AMZN,-0.2591
NVDA,0.3801
GOOGL,-0.7515
TSLA,0.1016
XOM,0.1453


In [89]:
in_sample_weights = (
    pd.concat([
        in_sample_weights_regularized,
        in_sample_weights_tangency,
        in_sample_weights_risk_parity,
        in_sample_weights_equal
    ], axis=1)
    .fillna(0)
)

in_sample_weights_scaled = in_sample_weights
in_sample_weights_scaled

,Regularized Weights,Tangency Weights,Risk Parity Weights,Equal Weights
AAPL,0.2373,0.3106,617.9459,0.1429
MSFT,0.3308,1.0731,925.5907,0.1429
AMZN,0.0472,-0.2591,537.3740,0.1429
NVDA,0.1968,0.3801,241.2644,0.1429
GOOGL,0.0114,-0.7515,713.2440,0.1429
TSLA,0.0902,0.1016,144.1950,0.1429
XOM,0.0864,0.1453,532.3875,0.1429


In [90]:
(   
    pmh.calc_summary_statistics(
        in_sample_assets_excess_returns @ in_sample_weights_scaled,
        annual_factor=12,
        provided_excess_returns=True,
        keep_columns=['Annualized mean', 'Annualized vol', 'Annualized sharpe']
    )
    .sort_values('Annualized Sharpe', ascending=False)
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
Tangency Weights,0.1089,0.1591,0.6845
Regularized Weights,0.0751,0.1253,0.5996
Equal Weights,0.0677,0.1253,0.5405
Risk Parity Weights,209.3088,410.0180,0.5105


In [91]:
(
    pmh.calc_summary_statistics(
        out_of_sample_assets_excess_returns @ in_sample_weights_scaled,
        annual_factor=12,
        provided_excess_returns=True,
        keep_columns=['annualized mean', 'annualized vol', 'annualized sharpe']
    )
    .sort_values('Annualized Sharpe', ascending=False)
)

,Annualized Mean,Annualized Vol,Annualized Sharpe
Regularized Weights,0.2339,0.1202,1.9455
Equal Weights,0.2204,0.1186,1.8580
Risk Parity Weights,625.5973,363.5650,1.7207
Tangency Weights,0.2780,0.2132,1.3043
